# Federated Iris Recognition - Client2
This notebook acts as **Client2** in the federated learning experiment using Flower.


In [ ]:
!pip install -q flwr tensorflow

In [1]:

import flwr as fl
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from model import build_model

# ==== Data configuration ====
TRAIN_DIR = r"C:\Users\priya\Desktop\Iris_Recognition\Final_Dataset\Clients\client2\train"
VAL_DIR   = r"C:\Users\priya\Desktop\Iris_Recognition\Final_Dataset\Clients\client2\val"
TEST_DIR  = r"C:\Users\priya\Desktop\Iris_Recognition\Final_Dataset\Clients\client2\test"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# No validation split needed
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_gen = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(
    TEST_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False
)

# Load from manually split folders
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Build model with correct number of output classes
NUM_CLASSES = train_generator.num_classes
model = build_model(NUM_CLASSES)


Found 148 images belonging to 74 classes.
Found 444 images belonging to 74 classes.
Found 148 images belonging to 74 classes.


In [3]:
import flwr as fl
from model import build_model
# ==== Flower client definition ====
class IrisClient(fl.client.NumPyClient):
    def __init__(self, model, train_gen, val_gen, test_gen):
        self.model = model
        self.train_gen = train_gen
        self.val_gen = val_gen
        self.test_gen = test_gen

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(
            self.train_gen,
            validation_data=self.val_gen,
            epochs=1,
            steps_per_epoch=len(self.train_gen),
            verbose=1
        )
        return self.model.get_weights(), len(self.train_gen), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, accuracy = self.model.evaluate(self.test_gen, verbose=0)
        return loss, len(self.test_gen), {"accuracy": float(accuracy)}

 #==== Start Flower client ====
fl.client.start_client(
    server_address="127.0.0.1:8081",
    client=IrisClient(model, train_generator, val_generator, test_gen).to_client()
)



	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      
INFO :      Received: train message 72f53654-fc31-48ed-a386-88beeda09fda


14/14 ━━━━━━━━━━━━━━━━━━━━ 52s 4s/step - accuracy: 0.0213 - loss: 4.4434 - val_accuracy: 0.2027 - val_loss: 3.5200


INFO :      Sent reply
INFO :      
INFO :      Received: train message 3370b969-3a9d-41b0-86b7-856da4c0351a


14/14 ━━━━━━━━━━━━━━━━━━━━ 53s 4s/step - accuracy: 0.1104 - loss: 3.9065 - val_accuracy: 0.3649 - val_loss: 3.1824


INFO :      Sent reply
INFO :      
INFO :      Received: train message fe750a37-7ffc-49f7-b6c4-b55b55562ccb


...

INFO :      Sent reply
INFO :      
INFO :      Received: train message e0522d09-aa25-4f71-931c-25cf6cb2fe65


14/14 ━━━━━━━━━━━━━━━━━━━━ 53s 4s/step - accuracy: 0.9522 - loss: 0.6527 - val_accuracy: 0.9122 - val_loss: 0.7940


INFO :      Sent reply
INFO :      
INFO :      Received: train message 4548fc9f-ef10-4875-9151-410484d218ea


14/14 ━━━━━━━━━━━━━━━━━━━━ 55s 4s/step - accuracy: 0.9793 - loss: 0.5691 - val_accuracy: 0.9392 - val_loss: 0.7360


INFO :      Sent reply
INFO :      
INFO :      Received: reconnect message 95923564-f2be-4c2a-89f3-c17606067689
INFO :      Disconnect and shut down


In [4]:
test_loss, test_acc = model.evaluate(test_gen, verbose=2)
print("\nTest accuracy:", test_acc)

5/5 - 5s - 904ms/step - accuracy: 0.9324 - loss: 0.7992

Test accuracy: 0.9324324131011963
